# 결과
num_words|None|5000|10000
---|---|---|---
ComplementNB|0.7649|0.7707|0.7707
LinearRegression|0.8134|0.8059|0.8077
LinearSVC|0.7774|0.7667|0.7814
DecisionTree|0.6211|0.6180|0.6202
RandomForest|0.6545|0.7012|0.6741
GradientBoosting|0.7703|0.7676|0.7667


# 회고
  - 딥러닝에 대해 부족한 점이 많음을 실감
    - 층을 쌓는 것에 대한 이해도 부족
    - 전달하는 인자들의 의미를 제대로 파악하지 못함
    - 층들이 하는 역할에 대해서도 이해하지 못함
    - 전반적으로 층에 대해 재학습이 필요함  
      
  - 층을 쌓다가 계속 오류가 발생해 지움  
    
  - Voting은 시간이 너무 걸려 일단 주석 처리함
    - 과제 제출 시한이 임박해서 누락시킴
    - 정확도가 가장 높았던 것 같지만 제출 기한을 넘길 수 없다고 판단
    - 다음에 딥러닝으로 구현할 때 이 부분도 같이 실행해보고자 함  
    
  
  - 과제가 그냥 노드를 합친 것에 지나지 않음
    - 과제 제출 때마다 부족함을 느끼게 됨
    - 잘 할 수 있을까 하는 생각을 계속하게 됨
    - 다음 과제는 좀더 열심히 해야지 하며 각오를 다짐
      
  - 다음 과제에서는 지금 부족한 부분을 좀더 채울 수 있기를 기대
  
  

In [40]:
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


In [41]:
def decoded_func(x):
    decoded = []
    
    for i in range(len(x)):
        t = ' '.join([index_to_word[index] for index in x[i]])
        
        decoded.append(t)
        
    return decoded

In [42]:
def data_align(num_words):

#    num_words = [None, 5000, 10000]
    
#    num_word = num_words[i]

    (x_train, y_train), (x_test, y_test) = \
        reuters.load_data(num_words=num_words, test_split=0.2)


    x_train = decoded_func(x_train)
    x_test = decoded_func(x_test)
    
    return x_train, y_train, x_test, y_test

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

from sklearn.naive_bayes import MultinomialNB
# 다항분포 나이브 베이즈 모델

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score  # 정확도 계산

In [44]:
def data_tfidf(x_train, x_test):
    dtmvector = CountVectorizer()
    x_train_dtm = dtmvector.fit_transform(x_train)

    tfidf_transformer = TfidfTransformer()
    tfidfv = tfidf_transformer.fit_transform(x_train_dtm)

    # 테스트 데이터를 DTM으로 변환
    x_test_dtm = dtmvector.transform(x_test)

    # DTM을 TF-IDF 행렬로 변환
    tfidfv_test = tfidf_transformer.transform(x_test_dtm)
    
    return tfidfv, tfidfv_test

In [45]:
def run_model(tfidfv, y_train, tfidfv_test):

    cb = ComplementNB()
    cb.fit(tfidfv, y_train)

    predicted = cb.predict(tfidfv_test)
    print('ComplementNB 정확도: ', accuracy_score(y_test, predicted))


    lr = LogisticRegression(C=10000, penalty='l2')
    lr.fit(tfidfv, y_train)

    predicted = lr.predict(tfidfv_test)
    print('LogisticRegression 정확도: ', accuracy_score(y_test, predicted))


    lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
    lsvc.fit(tfidfv, y_train)

    predicted = lsvc.predict(tfidfv_test)
    print('LinearSVC정확도: ', accuracy_score(y_test, predicted))


    tree = DecisionTreeClassifier(max_depth=10, random_state=0)
    tree.fit(tfidfv, y_train)

    predicted = tree.predict(tfidfv_test)
    print('DecisionTree 정확도: ', accuracy_score(y_test, predicted))


    forest = RandomForestClassifier(n_estimators=5, random_state=0)
    forest.fit(tfidfv, y_train)

    predicted = forest.predict(tfidfv_test)
    print('RandomForest 정확도: ', accuracy_score(y_test, predicted))


    grbt = GradientBoostingClassifier(random_state=0)
    grbt.fit(tfidfv, y_train)

    predicted = grbt.predict(tfidfv_test)
    print('GradientBoosting 정확도: ', accuracy_score(y_test, predicted))


#     voting_classifier = VotingClassifier(estimators=[
#         ('lr', LogisticRegression(C=10000, penalty='l2')),
#         ('cb', ComplementNB()),
#         ('grbt', GradientBoostingClassifier(random_state=0))
#     ], voting='soft', n_jobs=1)

#     voting_classifier.fit(tfidfv, y_train)

#     predicted = voting_classifier.predict(tfidfv_test)
#     print('Voting 정확도: ', accuracy_score(y_test, predicted))

In [46]:
word_index = reuters.get_word_index(path='reuters_word_index.json')
index_to_word = {index+3: word for word, index in word_index.items()}
for index, token in enumerate(('<pad>', '<sos>', '<unk>')):
    index_to_word[index] = token
    
# x_train, y_train, x_test, y_test = data_align()
# tfidfv, tfidfv_test = data_tfidf(x_train, x_test)
# #run_cb(tfidfv, y_train, tfidfv_test)
# run_model(tfidfv, y_train, tfidfv_test)

In [47]:
num_words = [None, 5000, 10000]

for num_word in num_words:
    print('='*50 + '\n' + 'num_words = ' + str(num_word) + '\n')
    x_train, y_train, x_test, y_test = data_align(num_word)
    tfidfv, tfidfv_test = data_tfidf(x_train, x_test)
    #run_cb(tfidfv, y_train, tfidfv_test)
    run_model(tfidfv, y_train, tfidfv_test)
    
    print('\n\n\n')

num_words = None

ComplementNB 정확도:  0.7649154051647373


/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression 정확도:  0.813446126447017


/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC정확도:  0.7773820124666073
DecisionTree 정확도:  0.6211041852181657
RandomForest 정확도:  0.6544968833481746
GradientBoosting 정확도:  0.7702582368655387




num_words = 5000

ComplementNB 정확도:  0.7707034728406055


...

LinearSVC정확도:  0.7666963490650045
DecisionTree 정확도:  0.6179875333926982
RandomForest 정확도:  0.701246660730187
GradientBoosting 정확도:  0.767586821015138




num_words = 10000

ComplementNB 정확도:  0.7707034728406055


/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression 정확도:  0.8076580587711487


/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC정확도:  0.7813891362422084
DecisionTree 정확도:  0.6202137132680321
RandomForest 정확도:  0.674087266251113
GradientBoosting 정확도:  0.7666963490650045




